In [2]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import importlib
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, lesioned_sim, compute_prob_tensor, init_params_tensor, num_stab_help
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

<module 'granch_utils.lesioned_sim' from '/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/lesioned_sim.py'>

In [8]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)
importlib.reload(init_params_tensor)
importlib.reload(compute_prob_tensor)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 20, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 20, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 20, 
    "grid_epsilon_start": 0.000001, "grid_epsilon_end": 1, "grid_epsilon_step": 20, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 0,  
    "V_prior": 1, 
    "alpha_prior": 3, 
    "beta_prior": 1, 
    "epsilon": 0.0001, "mu_epsilon":0.0001, "sd_epsilon": 0.0001, 
    "hypothetical_obs_grid_n": 10, 
    "world_EIGs": 0.0001, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(pair_each_stim = 1, n_feature=1)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan)
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])
pd.set_option('display.max_rows', None)

print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)


torch.Size([20, 20, 20])
torch.Size([10, 20, 20, 20])
torch.Size([20, 20, 20])
torch.Size([10, 20, 20, 20])
torch.Size([20, 20, 20])
torch.Size([10, 20, 20, 20])
torch.Size([20, 20, 20])
torch.Size([10, 20, 20, 20])
torch.Size([20, 20, 20])
torch.Size([10, 20, 20, 20])
torch.Size([20, 20, 20])
torch.Size([10, 20, 20, 20])
    stimulus_id  EIG Look_away
0             0 -0.0      True
1             1 -0.0      True
2             2 -0.0      True
3             3 -0.0      True
4             4  0.0      True
5             5  0.0      True
6           NaN  NaN       NaN
7           NaN  NaN       NaN
8           NaN  NaN       NaN
9           NaN  NaN       NaN
10          NaN  NaN       NaN
11          NaN  NaN       NaN
12          NaN  NaN       NaN
13          NaN  NaN       NaN
14          NaN  NaN       NaN
15          NaN  NaN       NaN
16          NaN  NaN       NaN
17          NaN  NaN       NaN
18          NaN  NaN       NaN
19          NaN  NaN       NaN
20          NaN  NaN     

/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/init_model_tensor.py:280: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.1458208013756522' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()


In [103]:
test_stim =  tensor_stimuli[0].stimuli_sequence
test_stim

{0: tensor([0.2030], dtype=torch.float64),
 1: tensor([0.2030], dtype=torch.float64),
 2: tensor([0.2030], dtype=torch.float64),
 3: tensor([0.2030], dtype=torch.float64),
 4: tensor([0.2030], dtype=torch.float64),
 5: tensor([0.2030], dtype=torch.float64)}

KeyboardInterrupt: 

In [77]:
import torch

# Create tensors A and B
A = torch.tensor([1, 2, 3])
B = torch.tensor([4, 5, 6])
C = torch.tensor([7, 8, 9])

# Create all combinations using torch.meshgrid()
grid_A, grid_B, grid_C= torch.meshgrid(A,B,C)

# Stack the grids to form tensor C
D = torch.stack((grid_A, grid_B, grid_C), dim=-1)

#print(D)

# Reshape tensor C to 2D if needed
D = D.view(-1, 3)

# Print the resulting tensor C
print(D.size())


torch.Size([27, 3])


RuntimeError: torch.meshgrid: Expected 0D or 1D tensor in the tensor list but got:  3  5
 3  6
 4  5
 4  6
[ torch.LongTensor{4,2} ]